## Course Tuning
### 23

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import lightgbm as lgb
import optuna.integration.lightgbm as lgb
from sklearn import datasets
from sklearn.model_selection import train_test_split
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

### 管理番号

In [2]:
model_No = 32
sample_No = 1

### sub-modelを使用するかどうか

In [3]:
use_sub_model = False

In [4]:
if use_sub_model:
    ALL_MERGE = 'intermediate/{}/all_merge_{}_{}_sub.f'.format(model_No, model_No, sample_No)
else:
    ALL_MERGE = 'intermediate/{}/all_merge_{}_{}.f'.format(model_No, model_No, sample_No)

In [5]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 584)

In [6]:
if use_sub_model:
    all_pitch['predict_curve'] = all_pitch['predict_curve'] / all_pitch['predict_straight']
    all_pitch['predict_slider'] = all_pitch['predict_slider'] / all_pitch['predict_straight']
    all_pitch['predict_shoot'] = all_pitch['predict_shoot'] / all_pitch['predict_straight']
    all_pitch['predict_fork'] = all_pitch['predict_fork'] / all_pitch['predict_straight']
    all_pitch['predict_changeup'] = all_pitch['predict_changeup'] / all_pitch['predict_straight']
    all_pitch['predict_sinker'] = all_pitch['predict_sinker'] / all_pitch['predict_straight']
    all_pitch['predict_cutball'] = all_pitch['predict_cutball'] / all_pitch['predict_straight']
    all_pitch.drop(columns=[
        'predict_0', 'predict_1', 'predict_2', 'predict_3', 'predict_4', 'predict_5', 'predict_6',
        'predict_7', 'predict_8', 'predict_9', 'predict_10', 'predict_11', 'predict_12',
       'predict_straight'
    ], inplace=True)

### train
- 行数: 257117

In [7]:
train = all_pitch.dropna(subset=['course'])
train.shape

(257117, 584)

#### 特徴量に加えた部分を抜く

In [8]:
if sample_No == 1:
    train = train.query('index > 60000')
elif sample_No == 2:
    train = train.query('index <= 60000 | index > 120000')
elif sample_No == 3:
    train = train.query('index <= 120000 | index > 180000')
elif sample_No == 4:
    train = train.query('index <= 180000')
train.shape

(197116, 584)

In [9]:
del all_pitch
gc.collect()

36

In [10]:
train_d = train.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

### LGBM学習
- multiclass : クラス数=13

In [11]:
X_train, X_test, y_train, y_test = train_test_split(train_d, train['course'])
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [12]:
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'dart',
        'metric' : 'multi_logloss',
        'num_class' : 13,
    }

In [13]:
%%time
best_params, tuning_history = dict(), list()
lgb_model = lgb.train(lgb_param, lgb_train,
                      valid_sets=lgb_eval,
                      verbose_eval=0,
                      best_params=best_params,
                      tuning_history=tuning_history)
print('Best Params:', best_params)
#print('Tuning history:', tuning_history)

/home/yasunobu_umehara/anaconda3/lib/python3.7/site-packages/optuna/_experimental.py:90: ExperimentalWarning: train is experimental (supported from v0.18.0). The interface can change in the future.
  ExperimentalWarning,
tune_num_leaves, val_score: 2.340094: 100%|##########| 20/20 [4:27:41<00:00, 803.05s/it]
tune_bagging_fraction_and_bagging_freq, val_score: 2.340094:  10%|#         | 1/10 [04:01<36:12, 241.43s/it][I 2020-06-20 11:41:17,194] Finished trial#0 with value: 2.340743080371187 with parameters: {'bagging_fraction': 0.5468883268292501, 'bagging_freq': 5}. Best is trial#0 with value: 2.340743080371187.
tune_bagging_fraction_and_bagging_freq, val_score: 2.340094:  20%|##        | 2/10 [07:51<31:45, 238.17s/it][I 2020-06-20 11:45:07,747] Finished trial#1 with value: 2.3411285439937677 with parameters: {'bagging_fraction': 0.48199021911149237, 'bagging_freq': 2}. Best is trial#0 with value: 2.340743080371187.
tune_bagging_fraction_and_bagging_freq, val_score: 2.339638:  30%|###   

tune_bagging_fraction_and_bagging_freq, val_score: 2.339638:  50%|#####     | 5/10 [20:31<20:28, 245.68s/it][I 2020-06-20 11:57:47,225] Finished trial#4 with value: 2.3412784218432985 with parameters: {'bagging_fraction': 0.5643450247886601, 'bagging_freq': 6}. Best is trial#2 with value: 2.339637753186007.
tune_bagging_fraction_and_bagging_freq, val_score: 2.339638:  60%|######    | 6/10 [24:23<16:06, 241.73s/it][I 2020-06-20 12:01:39,754] Finished trial#5 with value: 2.3414092207602843 with parameters: {'bagging_fraction': 0.494381410126377, 'bagging_freq': 2}. Best is trial#2 with value: 2.339637753186007.
tune_bagging_fraction_and_bagging_freq, val_score: 2.339638:  70%|#######   | 7/10 [28:10<11:51, 237.16s/it][I 2020-06-20 12:05:26,257] Finished trial#6 with value: 2.3418002089831482 with parameters: {'bagging_fraction': 0.44403106437804074, 'bagging_freq': 4}. Best is trial#2 with value: 2.339637753186007.
tune_bagging_fraction_and_bagging_freq, val_score: 2.339638:  80%|#######

tune_lambda_l1_and_lambda_l2, val_score: 2.338101:  75%|#######5  | 15/20 [1:13:20<24:33, 294.75s/it][I 2020-06-20 13:59:22,687] Finished trial#14 with value: 2.339398237368972 with parameters: {'lambda_l1': 0.1781478469894777, 'lambda_l2': 0.008946174782520931}. Best is trial#6 with value: 2.3381010594489195.
tune_lambda_l1_and_lambda_l2, val_score: 2.338101:  80%|########  | 16/20 [1:18:10<19:32, 293.19s/it][I 2020-06-20 14:04:12,222] Finished trial#15 with value: 2.3395578234001295 with parameters: {'lambda_l1': 0.31624751408262525, 'lambda_l2': 0.6431289078086456}. Best is trial#6 with value: 2.3381010594489195.
tune_lambda_l1_and_lambda_l2, val_score: 2.338101:  85%|########5 | 17/20 [1:23:29<15:02, 300.87s/it][I 2020-06-20 14:09:31,028] Finished trial#16 with value: 2.3382678007773983 with parameters: {'lambda_l1': 8.032580295707369, 'lambda_l2': 0.03896291205792476}. Best is trial#6 with value: 2.3381010594489195.
tune_lambda_l1_and_lambda_l2, val_score: 2.338101:  90%|#########

Best Params: {'lambda_l1': 8.074719414659954, 'lambda_l2': 1.5919119266007067, 'num_leaves': 6, 'feature_fraction': 0.516, 'bagging_fraction': 0.7965160701163017, 'bagging_freq': 5, 'min_child_samples': 20}
CPU times: user 8d 19h 41min 30s, sys: 31min 18s, total: 8d 20h 12min 48s
Wall time: 8h 50min 55s


### Feature Importance

In [14]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(30)

,feat_name,feat_imp
520,mul_bc_c_course_12,303
571,ave_bc_c_low_ball,305
367,mul_center_str,306
390,ave_center_str,311
364,mul_low_ball,317
356,mul_course_9,324
379,ave_course_9,324
357,mul_course_10,325
8,bat_order,329
31,ave_elapsed_game,334


### dart(32)
`
Best Params: {
    'lambda_l1': 8.074719414659954, 
    'lambda_l2': 1.5919119266007067, 
    'num_leaves': 6, 
    'feature_fraction': 0.516, 
    'bagging_fraction': 0.7965160701163017, 
    'bagging_freq': 5, 
    'min_child_samples': 20
}
Wall time: 8h 50min 55s
`

### Course Tuning結果(23)
`
Best Params: {
    'lambda_l1': 5.4143376259630465, 
    'lambda_l2': 0.006926732664484028, 
    'num_leaves': 4, 
    'feature_fraction': 0.41600000000000004, 
    'bagging_fraction': 0.9117189906923387, 
    'bagging_freq': 4, 
    'min_child_samples': 25
}
`
Wall time: 1h 56min 20s

### Course Tuning結果(12)
`
Best Params: {
    'lambda_l1': 8.769293390201968, 
    'lambda_l2': 3.913949617576324e-05, 
    'num_leaves': 6, 
    'feature_fraction': 0.4, 
    'bagging_fraction': 0.8391111798378441, 
    'bagging_freq': 4, 
    'min_child_samples': 50
}
`
Wall time: 2h 58min 40s